In [33]:
!pip install mysql-connector-python

In [34]:
import mysql.connector as c
from datetime import date

In [35]:
con = c.connect(host = "localhost", user = "root", passwd = "root",database="project1")

if con.is_connected():
    print("Successfully Connected ...")
else:
    print("Connection Failed...")

Successfully Connected ...


In [36]:
# creating database_cursor to perform SQL operation
db_cursor =con.cursor()

In [37]:
#Get database table
db_cursor.execute("SHOW TABLES")
for table in db_cursor:
	print(table)

('bills',)
('items',)
('login',)
('transaction',)


In [44]:
##### Display the report menu
def report_menu():
    while True:
        clear()
        print('##### REPORT MENU ####')
        print('-'*100)
        print('1.   Item List')
        print('2.   Sell Between Dates')
        print('3.   Last Bill information')
        print('4.   Amount collected')
        print('5.   Back to main Menu')
        choice = int(input('\nEnter your choice : '))
        if choice==1:
            item_list()
        if choice==2:
            date_wise_sell()
        if choice==3:
            last_bill_no()
        if choice==4:
            amount_collected()
        if choice==5:
            break
            

def search_menu():
    while True:
        clear()
        print('######## SEARCH MENU ####### ')
        print('-'*100)
        print('1.   Item Name')
        print('2.   Customer information')
        print('3.   Bill information')
        print('4.   Back to main Menu')
        choice = int(input('\nEnter your choice : '))
        if choice==1:
            search_item()
        if choice==2:
            search_customer()
        if choice==3:
            search_bill()
        if choice==4:
            break
    
def search_item():
    clear()
    item_name =input('Enter Item Name :')
    sql ='select * from items where item_name like "%{}%";'.format(item_name)
    db_cursor.execute(sql)
    records = db_cursor.fetchall()
    clear()
    print('Item Names start with :',item_name)
    print('-'*100)
    print('{:10s} {:30s} {:20s}'.format('Item ID','Item Name','Item Price'))
    print('-'*100)
    for row in records:
        print('{:<10d} {:30s} {:.2f}'.format(row[0],row[1],row[2]))
    print('-'*100)
    print('\nPress any key to continue....')
    wait= input()
    
    
#function to find out bill information
def search_bill():
    clear()
    bill_no = input('Enter Bill Number :')
    #SQL Query to take bill information from all tables.
    sql = 'select b.bill_id,b.name,b.phone,b.bill_date,t.item_id,t.qty,i.item_name,i.price from bills b,transaction t,items i \
           where b.bill_id = t.bill_id AND t.item_id= i.id and b.bill_id ={};'.format(bill_no)
    db_cursor.execute(sql) 
    records = db_cursor.fetchall()
    n = db_cursor.rowcount
    clear()
    print("Bill No :",bill_no)
    print('-'*100)
    if n<=0:
        print('Bill number {} does not exists'.format(bill_no))
    else:
        print('Customer Name : {}  Phone No :{}'.format(records[0][1],records[0][2]))
        print('Bill Date : {}'.format(records[0][3]))
        print('-'*100)
        print('{:10s} {:30s} {:20s} {:10s} {:30s}'.format('ID','Item Name','Qty','Price','Amount'))
        print('-'*100)
        for row in records:
            print('{:<10d} {:30s} {:<20d} {:.2f} {:>.2f}'.format(row[4],row[6],row[5],row[7],row[5]*row[7]))
        print('-'*100)
    print('\nPress any key to continue....')
    wait = input()
      
    
def search_customer():
    clear()
    cust_name =input('Enter customer Name :')
    #SQL query to take information from table.
    sql ='select * from bills where name like "%{}%";'.format(cust_name)
    db_cursor.execute(sql)
    records = db_cursor.fetchall()
    clear()
    print('Customer Names started with :',cust_name)
    print('-'*100)
    print('{:10s} {:30s} {:20s} {:20s}'.format('Bill No','Customer Name','Phone No','Bill Date'))
    print('-'*100)
    for row in records:
        print('{:<10d} {:30s} {:20s} {:20s}'.format(row[0],row[1],row[2],str(row[3])))
    print('-'*100)
    print('\nPress any key to continue....')
    wait= input()

    
#  Function to display amount collected between two dates
def amount_collected():
    clear()
    start_date = input('Enter start Date (yyyy-mm-dd) :')
    end_date   = input('Enter End   Date (yyyy-mm-dd) :')
    clear()
    print('Amount collected between: {} and {}'.format(start_date,end_date))
    print('-'*100)
    sql = 'select sum(t.qty*i.price) from bills b,transaction t,items i where b.bill_date between "{}" AND "{}" AND\
    b.bill_id = t.bill_id and t.item_id = i.id;'.format(start_date,end_date)
    db_cursor.execute(sql)
    result = db_cursor.fetchone()
    print(result)
    print('\nPress any key to continue.....')
    wait= input()
    
    

#   Create a report on date wise sell.
def date_wise_sell():
    clear()
    print('Date wise sell')
    print('-'*100)
    start_date = input('Enter start Date (yyyy-mm-dd) :')
    end_date   = input('Enter End Date (yyyy-mm-dd) :')
    sql = 'select * from bills where bill_date between "{}" and "{}"'.format(start_date,end_date)
    db_cursor.execute(sql)
    records = db_cursor.fetchall()
    clear()
    print('Bill No       Customer Name         Phone No         Bill Date')
    print('-'*100)
    for row in records:
        print('{:10d} {:30s} {:20s} {}'.format(row[0],row[1],row[2],row[3]))
    print('-'*100)
    print('\n\nPress any key to continue....')
    wait= input()
    
    
    
# generate bill 
def billing():
    clear()
    items =[]
    bill_no = last_bill_no()
    if bill_no[0]==None:
        bill_no =1
    else:
        bill_no = bill_no[0]+1 

    name = input('Enter customer Name :')
    phone = input('Enter Phone No :')
    t_date  = date.today()
    while True:
        no = int(input('Enter item No (press 0 to stop) :'))
        if no <=0:
            break
        else:
            item = find_item(no)
            if item==None:
                print('Item Not found  ')
            else:
                qty = int(input('Enter Item Qty :'))
                item = list(item)
                item.append(qty)
                items.append(item)

    clear()
    print(' -------------------Jejurkar Super Store-------------------------- ')
    print('                           wakad     ')
    print('            Phone:+91-7499190295, Email: jejurkaraniln72@gmail.com ')
    print('Bill No :{}                                    Date :{}'.format(bill_no,t_date))
    print('-'*100)
    print('Customer Name :{}                     Phone No :{}'.format(name,phone))
    print('-'*100)
    print('Item Id     Item Name                Price         Qty        Amount ')
    print('-'*100)
    total =0
    for i in items:
        print('{:<10d} {:25s} {:.2f}   {:^10d}     {:>.2f}'.format(i[0],i[1],i[2],i[3],i[2]*i[3]))
        total = total +i[2]*i[3]
    print('-'*100)
    print('Total Payable amount : {}'.format(total))
    print('\nPress any key to continue........')
    #insert data into tables
    sql ='insert into bills(name,phone,bill_date) values("{}","{}","{}");'.format(name,phone,t_date)
    db_cursor.execute(sql)    
    for i in items:
        sql='insert into transaction(item_id,qty,bill_id) values({},{},{});'.format(i[0],i[3],bill_no)
        db_cursor.execute(sql)
    wait= input()
    
    

# display all the items in items tables
def item_list():
    clear()
    sql="select * from items"
    db_cursor.execute(sql)
    records = db_cursor.fetchall()
    for row in records:
        print(row)
    print('\nPress any key to continue.....')
    wait = input()
    
    
    
#  change details in items table
def modify_item():
    clear()
    print('Modify Item ')
    print('-'*100)
    item_id = input('Enter Item ID :')
    item_name = input('Enter new Item Name :')
    item_price = input('Enter Item Price :')
    sql = 'update items set item_name = "{}", price ={} where id={}'.format(item_id,item_name,item_price)
    db_cursor.execute(sql)
    print('\n\nRecord Updated Successfully............')
    
    
def add_item():
    clear()
    print('Add New Item  ')
    print('-'*100)
    item_name = input('Enter new Item Name :')
    item_price = input('Enter Item Price :')
    sql = 'select * from items where item_name like "%{}%"'.format(item_name)
    db_cursor.execute(sql)
    record=db_cursor.fetchone()
    if record==None:
        sql = 'insert into items(item_name,price) values("{}",{});'.format(item_name,item_price)
        db_cursor.execute(sql)
        print('\n\nNew Item added successfully.....\nPress any key to continue....')
    else:
        print('\n\nItem Name already Exist.....\nPress any key to continue....')
    wait= input()
    
    
    
def find_item(no):
    db_cursor.execute('select * from items where id ={}'.format(no))
    record = db_cursor.fetchone()
    return record


def last_bill_no():
    db_cursor.execute('select max(bill_id) from bills')
    record = db_cursor.fetchone()
    return record

def clear():
    for _ in range(65):
        print()
        
        
def main_menu():
    while True:
        clear()
        print(' ***  MAIN  MENU  *** ')
        print('-'*100)
        print('1.   Add New Item')
        print('2.   Modify Item')
        print('3.   Billing')
        print('4.   Search Menu')
        print('5.   Report Menu')
        print('6.   Exit')
        choice = int(input('\n\nEnter your choice : '))
        if choice==1:
            add_item()
        if choice==2:
            modify_item()
        if choice==3:
            billing()
        if choice==4:
            search_menu()
        if choice==5:
            report_menu()
        if choice==6:
            break


if __name__=="__main__":
    clear()
    main_menu()



































































































































 ***  MAIN  MENU  *** 
----------------------------------------------------------------------------------------------------
1.   Add New Item
2.   Modify Item
3.   Billing
4.   Search Menu
5.   Report Menu
6.   Exit


Enter your choice : 3

































































Enter customer Name :sager
Enter Phone No :87684365
Enter item No (press 0 to stop) :1
Enter Item Qty :1
Enter item No (press 0 to stop) :4
Enter Item Qty :2
Enter item No (press 0 to stop) :0

































































 -------------------Jejurkar Super Store-------------------------- 
                           wakad     
            Phone:+91-7499190295, Email: jejurkaraniln72@gmail.com 
Bill No :8                                    Date :2022-12-18
----------------------------------------------------------------------

KeyboardInterrupt: 